集成算法的目的是将几个基估计器的预测与给定的学习算法结合起来，以提高单个估计器的通用性和鲁棒性。

集成方法一般分为两种：

- 在平均法（averaging methods)中，该方法的原理是构建多个独立的估计器，然后`取预测结果的平均`。一般来说，组合之后的估计器是会比单个估计器要好，因为`方差减小`了。

   - 示例: Bagging methods, Forests of randomized trees, …

- 相反，在提升法(boosting methods)中，`基估计器是按顺序建立的，降低了偏差`。其动机是将几个弱模型结合起来，形成一个强大的整体。

    - 示例: AdaBoost, Gradient Tree Boosting, …

## Bagging

在集成算法中， bagging方法形成了一类算法，它在原始训练集的随机子集上建立几个黑箱估计器的实例，然后将它们的个体预测聚合起来，形成最终的预测。

这些方法通过在基本估计器(例如决策树)的构造过程中`引入随机化`，然后`将其集成起来`，从而`降低单个基本估计器(如决策树)的方差`。

在许多情况下，bagging方法是一个非常简单的方法可以用来改进相对单一模型，而不需要调整底层。由于bagging方法提供了一种减少过度拟合的途径，因此**对强大模型和复杂模型(例如，充分生长的决策树)最有效**，与之对比的提升法在弱模型(例如浅层决策树)上表现最好。

![20220712152323](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220712152323.png)

In [1]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=0.5)

### 随机森林 -- RandomForestClassifier

在随机森林中(RandomForestClassifier和RandomForestRegressor类)， 集成模型中的每棵树构建时的样本都是由`训练集经过有放回抽样`(比如a bootstrap sample自助式采样法)得来。

在构建树的过程中进行结点分割时，选择的分割点是所有特征的最佳分割点，或特征的大小为 max_features 的随机子集的最佳分割。

这两种随机的目的是`降低森林估计器的方差`。事实上，单个决策树通常表现出很高的方差，并且往往会过拟合。在森林中注入随机性产生的决策树具有一定的解耦预测误差(decoupled prediction errors)。通过取这些预测的平均值，可以抵消掉一些误差。

`随机森林通过组合不同的树来减少方差，有时以增加一点点偏差为代价`。在实践中，方差减少通常是值得关注的，因此产生了一个整体更好的模型。

**scikit-learn实现通过平均它们的概率预测来组合分类器，而不是让每个分类器为单个类别进行投票。**

In [2]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston,load_iris

li = load_iris()
features2 = li.feature_names
x_cls ,y_cls = li.data,li.target
x_cls = pd.DataFrame(x_cls,columns=features2)
y_cls = pd.DataFrame(y_cls,columns=['target'])

from sklearn.model_selection import train_test_split
X_cls_train,X_cls_test,y_cls_train,y_cls_test = train_test_split(x_cls,y_cls,random_state=2022,test_size=0.2)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 默认bootstrap=True,有放回抽样，使用部分数据集
rf_cls = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=3,max_samples=0.8,bootstrap=True,random_state=2022)
rf_cls.fit(X_cls_train,y_cls_train)
rf_cls.score(X_cls_train,y_cls_train)

print(accuracy_score(y_cls_test,rf_cls.predict(X_cls_test)))
rf_cls.predict_proba(X_cls_test[:5])

0.9666666666666667


array([[0.        , 0.01675679, 0.98324321],
       [0.        , 0.01264354, 0.98735646],
       [1.        , 0.        , 0.        ],
       [0.        , 0.02520051, 0.97479949],
       [1.        , 0.        , 0.        ]])

注意：

控制树大小的参数的默认值(例如max_depth, min_samples_leaf等)会导致完全生长和未修剪的树，在某些数据集上可能树会非常大。为了减少内存消耗，应通过设置这些参数值来`控制树的复杂性和大小`。

在每次分割时，特性总是随机地进行分配。因此，即使训练数据相同，max_features=n_features, bootstrap=False，在搜索最佳分割的过程中，如果枚举的几个分割对准则的改进相同，那么找到的最佳分割也会有所不同。为了在拟合过程中获得确定性行为，必须确定random_state。

### 极端随机树 -- ExtraTreesClassifier

在极端随机树(参见 ExtraTreesClassifier 和 ExtraTreesRegressor 类)中，计算分割点方法中的`随机性进一步增强`。

与随机森林中一样，使用了候选特征的随机子集，但不像随机森林中是寻找最具区分度的阈值，而是`对每个候选特征随机绘制阈值，并选择这些随机生成的阈值中最佳的作为分割规则`。这种做法通常能够减少一点模型的方差，代价则是略微地增大偏差

In [4]:
from sklearn.ensemble import ExtraTreesClassifier

# 默认bootstrap = False，用整个数据集。
# 毕竟极端随机树，特征分裂阈值是随机生成的，随机性毕竟大了，所以没必要样本随机取了。
ef_cls = ExtraTreesClassifier(n_estimators=100,criterion='gini',max_depth=3,bootstrap=False,random_state=2022)
ef_cls.fit(X_cls_train,y_cls_train)
ef_cls.score(X_cls_train,y_cls_train)

print(accuracy_score(y_cls_test,ef_cls.predict(X_cls_test)))
ef_cls.predict_proba(X_cls_test[:5])

0.9666666666666667


array([[0.0052545 , 0.24071436, 0.75403113],
       [0.        , 0.10228781, 0.89771219],
       [0.93719823, 0.05370412, 0.00909765],
       [0.01406477, 0.33230632, 0.65362891],
       [0.95267728, 0.03891332, 0.0084094 ]])

### 参数

使用这些方法时要调整的参数主要是 n_estimators 和 max_features。

**n_estimators**是森林里树的数量，通常数量越大，效果越好，但是计算时间也会随之增加。此外要注意，当树的数量超过一个临界值之后，算法的效果并不会很显著地变好。

**max_features**是分割节点时考虑的特征的随机子集的大小。`这个值越低，方差减小得越多，但是偏差的增大也越多`。根据经验:

- **回归问题**使用 `max_features = None` （考虑所有特征）；max_depth = None 和 min_samples_split = 2结合通常会有不错的效果（即完全生长的树）；
- **分类问题**使用 `max_features = 'sqrt'` （随机考虑 sqrt(n_features) 特征）是比较好的默认值；

以上这些（默认）值通常不是最佳的，同时还可能消耗大量的内存，最佳参数值应由`交叉验证`获得。 


另外，在随机森林中，默认使用自助采样法（bootstrap = True）， 然而 extra-trees 的默认策略是使用整个数据集（bootstrap = False）。 当使用自助采样法方法抽样时，泛化精度是可以通过剩余的或者袋外(out-of-bag) 的样本来评估的，设置 oob_score = True 即可实现。


注意：默认参数下模型复杂度是：O(M*N*log(N)) ， 其中 M 是树的数目， N 是样本数。 可以通过设置以下参数来`降低模型复杂度`： 
- `min_samples_split`（分割内部节点的最小样本数） ；
- `max_leaf_nodes`（最大叶子节点的数量） ；
- `max_depth`（最大深度）  ；
- `min_samples_leaf`（一个叶子节点所需的最小样本数）；

## 特征重要性评估

特征对目标变量预测的相对重要性可通过特征使用的相对顺序（即深度）进行评估。在scikit-learn中，一个特征所贡献的样本比例与拆分后的不纯度的减少相结合，形成了对该特征预测能力的标准化估计。
